<a href="https://colab.research.google.com/github/sagarpyakurel/MinCut-and-Max-Flow/blob/main/460_Graphs_Flows_and_Cuts_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Test graph, expect max flow = 11
G = [#  A   B   C   D   E
     [  0, 20,  0,  0,  0], # A
     [  0,  0,  5,  6,  0], # B
     [  0,  0,  0,  3,  7], # C
     [  0,  0,  0,  0,  8], # D
     [  0,  0,  0,  0,  0]  # E
]

In [ ]:

import copy # to make deep copy

def find_path(graph, source, target, path):
  """Finds a path between two vertices in a directed acyclic graph.

  Inputs
  ------
  graph : list
    adjacency matrix of the graph to search
  source, target : int
    The two vertices at both ends of the path
  path : list
    path from previous recursive call; [] for initial call

  Returns
  -------
  path : list
    vertices along the path from source to vertex; None if path doesn't exist.
  """
  # The source is always part of the path
  path.append(source)

  # Base case
  if source == target:
    return path

  # Consider every vertex adjacent to the source; because we use adjacency
  # matrix representation, we examine the row corresponding to the source
  # vertex: graph[source]. Every element in that row that is not zero represents
  # an edge. If the vertex on the other side of that edge is not already in the
  # path, we add it to the path. Then we ask if there is a parth from that
  # vertex to the target vertex.
  for v in range(len(graph)):
    if graph[source][v] > 0:
      if v not in path:
        possible_path = find_path(graph, v, target, path)
        if possible_path:
          return path
  return None

def find_min_cut(graph, residual_graph, source):
  """Finds the minimum cut of the graph after the max flow has been computed.

  Inputs
  ------
  graph : list
    adjacency matrix of the original graph
  residual_graph : list
    adjacency matrix of the residual graph after max flow computation
  source : int
    label of the source vertex

  Returns
  -------
  min_cut : list
    List of edges in the minimum cut, represented as tuples (u, v)
  """
  # Step 1: Identify all vertices reachable from the source in the residual graph
  visited = [False] * len(graph)
  queue = [source]
  visited[source] = True

  while queue:
    u = queue.pop(0)
    for v in range(len(residual_graph)):
      if residual_graph[u][v] > 0 and not visited[v]:
        queue.append(v)
        visited[v] = True

  # Step 2: Find all edges that cross from reachable set to non-reachable set
  min_cut = []
  for u in range(len(graph)):
    for v in range(len(graph)):
      if visited[u] and not visited[v] and graph[u][v] > 0:
        min_cut.append((u, v))

  return min_cut

def ff(graph, source, target):
  """Finds the maximum flow across a flow graph and identifies the minimum cuts
  that will disable the flow between a source and a target vertex in the graph.

  Inputs
  ------
  graph : list
    adjacency matrix of the input graph
  source : int
    label of the source vertex
  target : int
    label of the target vertex

  Returns
  -------
  max_flow : int
    The max flow can can travel from source to target in the input graph
  min_cuts : list
    The small set of edges that can reduce the graph's flow to 0.
  """

  # Variable to return
  max_flow = 0

  # Initially, the residual graph is identical to the input graph
  residual_graph = copy.deepcopy(graph)

  # Find an augmenting path; any such path is fine, to get the iteration loop
  # started. The loop continues as long as there is an augmenting path in the
  # residual graph. (Reminder: an augmenting path in the residual graph is a
  # path from the source to the target vertex).
  augmenting_path = find_path(residual_graph, source, target, [])

  while augmenting_path:

    # Find the edge with the least capacity along this path. Edges can be
    # identified by the elements of augmenting_path: adjacent elements of the
    # list are the vertices on that edge, e.g., an edge u --> v in the path will
    # have u = augmenting_path[i] and
    #      v = augmenting_path[i+1]
    # This is a standard search for a min value in a collection of values. It
    # begins by assuming that the first edge in the augmenting path has the least
    # capacity of the path. That edge is between the vertices in positions [0]
    # and [1] in the augmenting_path list. The weight of their edge is obtained
    # by referencing the residual_graph list for these two vertices. Then we
    # traverse the remaining edges in the augmenting path in to see if there is
    # one with less capacity.
    min_capacity = residual_graph[augmenting_path[0]][augmenting_path[1]]
    i = 1
    while i < len(augmenting_path)-1: # -1 to compensate for [i+1] below
      # Obtain vertices u, v for edge u --> v in augmenting path
      u = augmenting_path[i]
      v = augmenting_path[i+1]
      if residual_graph[u][v] < min_capacity:
        min_capacity = residual_graph[u][v]
      i += 1

    # Now that we know the bottleneck of the augmenting path, we can add its
    # capacity to the max flow of the graph, because that's as much flow as we
    # can push through this path.
    max_flow += min_capacity

    # Update residual capacities along the augmenting path. This is done by
    # traversing the augmenting path (yes, again), subtracting the path's
    # minimum capacity from its existing edges, and adding back-edges where
    # necessary based on the skew symmetry property.
    i = 0
    while i < len(augmenting_path)-1: # -1 to compensate for [i+1] below
      # Obtain vertices u, v for edge u --> v in augmenting path
      u = augmenting_path[i]
      v = augmenting_path[i+1]
      residual_graph[u][v] -= min_capacity # existing edge
      residual_graph[v][u] += min_capacity # back-edge due to skew symmetry
      i += 1

    # Now that the residual graph has been updated let's see if there is another
    # path between source and target. If None, the while loop ends.
    augmenting_path = find_path(residual_graph, source, target, [])

  # Find the minimum cut using the residual graph
  min_cuts = find_min_cut(graph, residual_graph, source)

  # Convert min_cut indices to vertex names
  vertex_names = ['A', 'B', 'C', 'D', 'E']
  min_cuts_named = [(vertex_names[u], vertex_names[v]) for u, v in min_cuts]

  # Done
  return max_flow, min_cuts_named

# Call the ff function to find the maximum flow and minimum cuts
max_flow, min_cuts = ff(G, 0, 4)

# Print the results
print("Max Flow:", max_flow)
print("Min Cuts:", min_cuts)

Max Flow: 11
Min Cuts: [('B', 'C'), ('B', 'D')]
